### <b style="color:blue;"> Gather Input Data </b>

In [0]:
%fs ls /FileStore/tables/ged

path,name,size,modificationTime
dbfs:/FileStore/tables/ged/SolarEdge_data_for_modelling.csv,SolarEdge_data_for_modelling.csv,3607239,1673709696000
dbfs:/FileStore/tables/ged/SolarEdge_data_for_streaming.csv,SolarEdge_data_for_streaming.csv,89278,1673709690000
dbfs:/FileStore/tables/ged/lab4/,lab4/,0,1669732281000
dbfs:/FileStore/tables/ged/weather_data_for_modelling.csv,weather_data_for_modelling.csv,3526145,1673709918000
dbfs:/FileStore/tables/ged/weather_data_for_streaming.csv,weather_data_for_streaming.csv,84313,1673709908000


In [0]:
# remove tables
#dbutils.fs.rm('dbfs:/FileStore/tables/ged/SolarEdge_ProductionWSplit.csv', True)

Out[1]: True

In [0]:
#import seaborn as sns
import pandas as pd
#import matplotlib as plt
from pyspark.sql.functions import *
from datetime import date, datetime, timedelta

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator 
from pyspark.ml.feature import VectorAssembler 
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [0]:
print(sc.version) # SparkContext version
print(sc.pythonVer) # Python version of Spark Context

3.2.1
3.8

#### <b style="color:blue;"> SolarEdge PV data </b>

Production, SelfConsumption, FeedIn

In [0]:
pv_df = spark.read.csv("dbfs:/FileStore/tables/ged/SolarEdge_data_for_modelling.csv", header=True, inferSchema=True)

In [0]:
#type(pv_df)

In [0]:
pv_df.printSchema() # Check the schema of columns

root
-- _c0: integer (nullable = true)
-- timestamp: timestamp (nullable = true)
-- production: double (nullable = true)

In [0]:
pv_df = pv_df.withColumnRenamed("timestamp","DateTime")
pv_df = pv_df.select("DateTime", "production")
pv_df = pv_df.withColumn("DateTime", date_trunc("hour", pv_df.DateTime))
pv_df = pv_df.groupby('DateTime').agg(sum('production').alias('production'))
pv_df = pv_df.sort("DateTime")

In [0]:
pv_df.printSchema() # Check the schema of columns

root
-- DateTime: timestamp (nullable = true)
-- production: double (nullable = true)

In [0]:
#display(pv_df.describe())

#### <b style="color:blue;"> Weather data </b>

In [0]:
smn_df = spark.read.csv("dbfs:/FileStore/tables/ged/weather_data_for_modelling.csv", header=True, inferSchema=True)

In [0]:
smn_df = smn_df.select("DateTime", "rr_SMA", "ss_SMA", "dd_SMA", "ff_SMA")
smn_df = smn_df.withColumn("DateTime", date_trunc("hour", smn_df.DateTime))
smn_df = smn_df.groupby(['DateTime']).agg(
    sum('rr_SMA').alias('rr_SMA'),\
    sum('ss_SMA').alias('ss_SMA'),\
    mean('dd_SMA').alias('dd_SMA'),\
    mean('ff_SMA').alias('ff_SMA'))
smn_df= smn_df.sort("DateTime")
smn_df.show(5)

+-------------------+------+------+-------------+------------+
 DateTime|rr_SMA|ss_SMA| dd_SMA| ff_SMA|
+-------------------+------+------+-------------+------------+
2019-07-01 00:00:00| 0.0| 0.0| 249.6666667| 2.516666667|
2019-07-01 01:00:00| 0.0| 0.0| 247.1666667| 9.2|
2019-07-01 02:00:00| 0.0| 0.0| 248.83333335|12.466666665|
2019-07-01 03:00:00| 0.0| 0.0| 227.5|6.9666666665|
2019-07-01 04:00:00| 0.0| 0.0|131.999999985| 3.7|
+-------------------+------+------+-------------+------------+
only showing top 5 rows

#### <b style="color:blue;"> Combine and prepare data </b>

In [0]:
data_df = pv_df.join(smn_df, ['DateTime'], 'left').sort("DateTime")

In [0]:
data_df.select([count(when(isnan(c), c)).alias(c) for c in data_df.drop('DateTime').columns]).show() # count NaNs

+----------+------+------+------+------+
production|rr_SMA|ss_SMA|dd_SMA|ff_SMA|
+----------+------+------+------+------+
 0| 0| 0| 0| 0|
+----------+------+------+------+------+

In [0]:
data_df.select([count(when(isnull(c), c)).alias(c) for c in data_df.drop('DateTime').columns]).show() # count Nulls

+----------+------+------+------+------+
production|rr_SMA|ss_SMA|dd_SMA|ff_SMA|
+----------+------+------+------+------+
 10858| 0| 0| 0| 0|
+----------+------+------+------+------+

In [0]:
data_df = data_df.fillna(0) # Replace missing values

In [0]:
#https://www.learntospark.com/2022/07/apache-spark-date-format-and-timestamp.html
data_df = data_df.withColumn('date', col('DateTime').cast('date'))
data_df = data_df.withColumn("year",year("DateTime")) # To Get Year from date or Time column
data_df = data_df.withColumn("month",month("DateTime"))
data_df = data_df.withColumn("day",dayofmonth("DateTime"))
data_df = data_df.withColumn("hour",hour("DateTime"))
data_df = data_df.withColumn("quarter-of-year",quarter("DateTime"))
data_df = data_df.withColumn("week-of-year",weekofyear("DateTime")) 

data_df.show(3)

+-------------------+----------+------+------+------------+------------+----------+----+-----+---+----+---------------+------------+
 DateTime|production|rr_SMA|ss_SMA| dd_SMA| ff_SMA| date|year|month|day|hour|quarter-of-year|week-of-year|
+-------------------+----------+------+------+------------+------------+----------+----+-----+---+----+---------------+------------+
2019-07-01 00:00:00| 0.0| 0.0| 0.0| 249.6666667| 2.516666667|2019-07-01|2019| 7| 1| 0| 3| 27|
2019-07-01 01:00:00| 0.0| 0.0| 0.0| 247.1666667| 9.2|2019-07-01|2019| 7| 1| 1| 3| 27|
2019-07-01 02:00:00| 0.0| 0.0| 0.0|248.83333335|12.466666665|2019-07-01|2019| 7| 1| 2| 3| 27|
+-------------------+----------+------+------+------------+------------+----------+----+-----+---+----+---------------+------------+
only showing top 3 rows

#### <b style="color:blue;"> Visualise data </b>

In [0]:
data_df.show(3)

+-------------------+----------+------+------+------------+------------+----------+----+-----+---+----+---------------+------------+
 DateTime|production|rr_SMA|ss_SMA| dd_SMA| ff_SMA| date|year|month|day|hour|quarter-of-year|week-of-year|
+-------------------+----------+------+------+------------+------------+----------+----+-----+---+----+---------------+------------+
2019-07-01 00:00:00| 0.0| 0.0| 0.0| 249.6666667| 2.516666667|2019-07-01|2019| 7| 1| 0| 3| 27|
2019-07-01 01:00:00| 0.0| 0.0| 0.0| 247.1666667| 9.2|2019-07-01|2019| 7| 1| 1| 3| 27|
2019-07-01 02:00:00| 0.0| 0.0| 0.0|248.83333335|12.466666665|2019-07-01|2019| 7| 1| 2| 3| 27|
+-------------------+----------+------+------+------------+------------+----------+----+-----+---+----+---------------+------------+
only showing top 3 rows

In [0]:
data_df.filter((data_df.month == 7) & (data_df.day ==1)).show(24)

+-------------------+-------------------+------+------+------------------+------------------+----------+----+-----+---+----+---------------+------------+
 DateTime| production|rr_SMA|ss_SMA| dd_SMA| ff_SMA| date|year|month|day|hour|quarter-of-year|week-of-year|
+-------------------+-------------------+------+------+------------------+------------------+----------+----+-----+---+----+---------------+------------+
2019-07-01 00:00:00| 0.0| 0.0| 0.0| 249.6666667| 2.516666667|2019-07-01|2019| 7| 1| 0| 3| 27|
2019-07-01 01:00:00| 0.0| 0.0| 0.0| 247.1666667| 9.2|2019-07-01|2019| 7| 1| 1| 3| 27|
2019-07-01 02:00:00| 0.0| 0.0| 0.0| 248.83333335| 12.466666665|2019-07-01|2019| 7| 1| 2| 3| 27|
2019-07-01 03:00:00| 0.0| 0.0| 0.0| 227.5| 6.9666666665|2019-07-01|2019| 7| 1| 3| 3| 27|
2019-07-01 04:00:00| 0.0| 0.0| 0.0| 131.999999985| 3.7|2019-07-01|2019| 7| 1| 4| 3| 27|
2019-07-01 05:00:00| 0.005| 0.0| 8.0| 153.66666665| 2.5|2019-07-01|2019| 7| 1| 5| 3| 27|
2019-07-01 06:00:00|0.33399999999999996| 0.0| 58.0| 178.5| 3.183333333|2019-07-01|2019| 7| 1| 6| 3| 27|
2019-07-01 07:00:00| 0.7320000000000001| 0.0| 60.0| 236.0| 7.0166666665|2019-07-01|2019| 7| 1| 7| 3| 27|
2019-07-01 08:00:00| 1.676| 0.0| 19.0| 248.83333335| 12.0|2019-07-01|2019| 7| 1| 8| 3| 27|
2019-07-01 09:00:00| 4.254| 0.0| 53.0| 207.3333333|13.149999999999999|2019-07-01|2019| 7| 1| 9| 3| 27|
2019-07-01 10:00:00| 6.106| 0.0| 60.0| 180.1666667|13.916666665000001|2019-07-01|2019| 7| 1| 10| 3| 27|
2019-07-01 11:00:00| 5.602| 0.0| 60.0| 187.0| 13.066666665|2019-07-01|2019| 7| 1| 11| 3| 27|
2019-07-01 12:00:00| 7.513| 0.0| 60.0| 184.6666667| 12.95|2019-07-01|2019| 7| 1| 12| 3| 27|
2019-07-01 13:00:00| 9.683| 0.0| 60.0| 199.33333335| 9.116666666499999|2019-07-01|2019| 7| 1| 13| 3| 27|
2019-07-01 14:00:00| 9.607| 0.0| 60.0| 195.83333335| 8.649999999999999|2019-07-01|2019| 7| 1| 14| 3| 27|
2019-07-01 15:00:00| 9.068999999999999| 0.0| 50.0| 223.1666667| 8.9333333315|2019-07-01|2019| 7| 1| 15| 3| 27|
2019-07-01 16:00:00| 7.336| 0.0| 60.0| 204.1666667| 11.566666665|2019-07-01|2019| 7| 1| 16| 3| 27|
2019-07-01 17:00:00| 5.513| 0.0| 45.0| 274.0| 11.3999999985|2019-07-01|2019| 7| 1| 17| 3| 27|
2019-07-01 18:00:00| 3.689| 0.0| 0.0| 257.6666667| 6.5333333335|2019-07-01|2019| 7| 1| 18| 3| 27|
2019-07-01 19:00:00| 0.8950000000000001| 0.0| 0.0| 177.66666665| 4.983333333|2019-07-01|2019| 7| 1| 19| 3| 27|
2019-07-01 20:00:00| 0.038| 0.0| 0.0| 237.0| 3.9666666665|2019-07-01|2019| 7| 1| 20| 3| 27|
2019-07-01 21:00:00| 0.008| 0.0| 0.0| 346.58333335| 7.074999999999999|2019-07-01|2019| 7| 1| 21| 3| 27|
2019-07-01 22:00:00| 0.0| 0.0| 0.0|124.33333331499999|6.6000000000000005|2019-07-01|2019| 7| 1| 22| 3| 27|
2019-07-01 23:00:00| 0.0| 1.6| 0.0|100.00000001500001| 10.083333335|2019-07-01|2019| 7| 1| 23| 3| 27|
+-------------------+-------------------+------+------+------------------+------------------+----------+----+-----+---+----+---------------+------------+
only showing top 24 rows

In [0]:
data_pddf_0701 = data_df.filter((data_df.month == 7) & (data_df.day ==1))

In [0]:
display(data_pddf_0701)

DateTime,production,rr_SMA,ss_SMA,dd_SMA,ff_SMA,date,year,month,day,hour,quarter-of-year,week-of-year
2019-07-01T00:00:00.000+0000,0.0,0.0,0.0,249.6666667,2.516666667,2019-07-01,2019,7,1,0,3,27
2019-07-01T01:00:00.000+0000,0.0,0.0,0.0,247.1666667,9.2,2019-07-01,2019,7,1,1,3,27
2019-07-01T02:00:00.000+0000,0.0,0.0,0.0,248.83333335,12.466666665,2019-07-01,2019,7,1,2,3,27
2019-07-01T03:00:00.000+0000,0.0,0.0,0.0,227.5,6.9666666665,2019-07-01,2019,7,1,3,3,27
2019-07-01T04:00:00.000+0000,0.0,0.0,0.0,131.999999985,3.7,2019-07-01,2019,7,1,4,3,27
2019-07-01T05:00:00.000+0000,0.005,0.0,8.0,153.66666665,2.5,2019-07-01,2019,7,1,5,3,27
2019-07-01T06:00:00.000+0000,0.33399999999999996,0.0,58.0,178.5,3.183333333,2019-07-01,2019,7,1,6,3,27
2019-07-01T07:00:00.000+0000,0.7320000000000001,0.0,60.0,236.0,7.0166666665,2019-07-01,2019,7,1,7,3,27
2019-07-01T08:00:00.000+0000,1.676,0.0,19.0,248.83333335,12.0,2019-07-01,2019,7,1,8,3,27
2019-07-01T09:00:00.000+0000,4.254,0.0,53.0,207.3333333,13.149999999999999,2019-07-01,2019,7,1,9,3,27


Output can only be rendered in Databricks

#### <b style="color:blue;"> Prepare data </b>

Add production previous hour and previous day with [lag function](https://sparkbyexamples.com/pyspark/pyspark-lag-function/)

In [0]:
window = Window.orderBy("DateTime")
data_df = data_df.withColumn("lag_1h",lag("production",1).over(window))
data_df = data_df.withColumn("lag_24h",lag("production",24).over(window))
data_df = data_df.na.drop() # drops data from 1st day

In [0]:
def train_test_split_date(df, split_col):
  """Calculate the date to split test and training sets"""
  max_date = df.agg({split_col: 'max'}).collect()[0][0] # latest date in dataset
  min_date = df.agg({split_col: 'min'}).collect()[0][0] # earliest date in dataset
  range_in_days = max_date - min_date
  split_date = max_date - timedelta(days=(range_in_days * 0.2).days) # testset = 20%
  return split_date

In [0]:
# Find the date to use in spitting test and train
split_date = train_test_split_date(data_df, 'date')

# Create Sequential Test and Training Sets
train_df = data_df.where(data_df['date'] < split_date) 
test_df = data_df.where(data_df['date'] >= split_date)

print("train_df rows/cols: ", train_df.count(), len(train_df.columns))
print("test_df rows/cols: ", test_df.count(), len(test_df.columns))

train_df rows/cols: 23376 15
test_df rows/cols: 5856 15

In [0]:
features_nolag = [i for i in list(data_df.columns) if i not in ['production', 'DateTime', 'date', 'day', 'lag_1h', 'lag_24h']] # also exclude target
features_wlag1h = [i for i in list(data_df.columns) if i not in ['production', 'DateTime', 'date', 'day', 'lag_24h']] # also exclude target
features_wlag24h = [i for i in list(data_df.columns) if i not in ['production', 'DateTime', 'date', 'day', 'lag_1h']] # also exclude target
features_wlags = [i for i in list(data_df.columns) if i not in ['production', 'DateTime', 'date', 'day']] # also exclude target

### <b style="color:blue;"> Models </b>

ML Pipeline

In [0]:
vec_assembler_nolag = VectorAssembler(inputCols=features_nolag, outputCol='features') # Feature Assembler
vec_assembler_wlag1h = VectorAssembler(inputCols=features_wlag1h, outputCol='features') # Feature Assembler
vec_assembler_wlag24h = VectorAssembler(inputCols=features_wlag24h, outputCol='features') # Feature Assembler
vec_assembler_wlags = VectorAssembler(inputCols=features_wlags, outputCol='features') # Feature Assembler

#data_df = vec.transform(data_df)

#### <b style="color:blue;"> Random Forest Regression</b>
[PySpark Reference](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.RandomForestRegressor.html#pyspark.ml.regression.RandomForestRegressor)

In [0]:
rf = RandomForestRegressor(featuresCol="features", labelCol="production", predictionCol="predicted_production", # predictionCol="predicted_production",   # FALLS NOTWENDIG, ÜBERALL ANPASSEN
                           seed=42 
                           ) 

evaluator = RegressionEvaluator(labelCol="production",  
                                predictionCol="predicted_production") # predictionCol="predicted_production")   # FALLS NOTWENDIG, ÜBERALL ANPASSEN

model_name = "rf" # for DF

In [0]:
pipeline_nolag = Pipeline(stages=[vec_assembler_nolag, rf])
rf_model_nolag = pipeline_nolag.fit(train_df)
rf_prediction_nolag = rf_model_nolag.transform(test_df)

In [0]:
#rf_prediction.head(1)

In [0]:
#display(rf_prediction.describe())

In [0]:
rf_rmse_nolag = evaluator.evaluate(rf_prediction_nolag, {evaluator.metricName: "rmse"}) 
rf_r2_nolag = evaluator.evaluate(rf_prediction_nolag, {evaluator.metricName: "r2"}) 
#print('RMSE: ' + str(rf_rmse_nolag)) 
#print('R^2: ' + str(rf_r2_nolag)) 

In [0]:
columns_DF = ['Model', 'Input Data', 'RMSE', 'R^2']
evaluation_DF = spark.createDataFrame([[model_name, 'no lag', rf_rmse_nolag, rf_r2_nolag]], columns_DF)

In [0]:
# lag 1h
pipeline_wlag1h = Pipeline(stages=[vec_assembler_wlag1h, rf])
rf_model_wlag1h = pipeline_wlag1h.fit(train_df)
rf_prediction_wlag1h = rf_model_wlag1h.transform(test_df)

rf_rmse_wlag1h = evaluator.evaluate(rf_prediction_wlag1h, {evaluator.metricName: "rmse"}) 
rf_r2_wlag1h = evaluator.evaluate(rf_prediction_wlag1h, {evaluator.metricName: "r2"}) 
#print('RMSE: ' + str(rf_rmse_wlag1h)) 
#print('R^2: ' + str(rf_r2_wlag1h)) 

newRow = spark.createDataFrame([[model_name, 'with lag 1 hour', rf_rmse_wlag1h, rf_r2_wlag1h]], columns_DF)
evaluation_DF = evaluation_DF.union(newRow)

In [0]:
# lag 24h
pipeline_wlag24h = Pipeline(stages=[vec_assembler_wlag24h, rf])
rf_model_wlag24h = pipeline_wlag24h.fit(train_df)
rf_prediction_wlag24h = rf_model_wlag24h.transform(test_df)

rf_rmse_wlag24h = evaluator.evaluate(rf_prediction_wlag24h, {evaluator.metricName: "rmse"}) 
rf_r2_wlag24h = evaluator.evaluate(rf_prediction_wlag24h, {evaluator.metricName: "r2"}) 
#print('RMSE: ' + str(rf_rmse_wlag24h)) 
#print('R^2: ' + str(rf_r2_wlag24h)) 

newRow = spark.createDataFrame([[model_name, 'with lag 24 hours', rf_rmse_wlag24h, rf_r2_wlag24h]], columns_DF)
evaluation_DF = evaluation_DF.union(newRow)


In [0]:
# lags
pipeline_wlags = Pipeline(stages=[vec_assembler_wlags, rf])
rf_model_wlags = pipeline_wlags.fit(train_df)
rf_prediction_wlags = rf_model_wlags.transform(test_df)

rf_rmse_wlags = evaluator.evaluate(rf_prediction_wlags, {evaluator.metricName: "rmse"}) 
rf_r2_wlags = evaluator.evaluate(rf_prediction_wlags, {evaluator.metricName: "r2"}) 
#print('RMSE: ' + str(rf_rmse_wlags)) 
#print('R^2: ' + str(rf_r2_wlags)) 

newRow = spark.createDataFrame([[model_name, 'with lags', rf_rmse_wlags, rf_r2_wlags]], columns_DF)
evaluation_DF = evaluation_DF.union(newRow)

In [0]:
tree = rf_model_wlag1h.stages[-1]
print(tree.featureImportances) # load feature importance from the model object

(10,[0,1,2,3,4,5,6,7,8,9],[7.034528289228866e-05,0.26791026184929434,0.0004171679569694342,0.01611318128254171,5.572484939467409e-05,0.005853006414274052,0.11810193096971482,0.0020048254464095482,0.02398324943914442,0.5654903065093648])

In [0]:
# Print the trees with nodes: 
print('Trees with Nodes: {}'.format(tree.toDebugString))

Trees with Nodes: RandomForestRegressionModel: uid=RandomForestRegressor_25f99bc303a5, numTrees=20, numFeatures=10
 Tree 0 (weight 1.0):
 If (feature 8 <= 39.5)
 If (feature 1 <= 17.5)
 If (feature 1 <= 0.5)
 If (feature 9 <= 3.8175)
 If (feature 6 <= 7.5)
 Predict: 0.008625918924595674
 Else (feature 6 > 7.5)
 Predict: 0.36396208530805685
 Else (feature 9 > 3.8175)
 If (feature 3 <= 4.36666666675)
 Predict: 3.041021739130435
 Else (feature 3 > 4.36666666675)
 Predict: 4.23403007518797
 Else (feature 1 > 0.5)
 If (feature 9 <= 3.8175)
 If (feature 1 <= 4.5)
 Predict: 1.2047225274725277
 Else (feature 1 > 4.5)
 Predict: 1.6006983240223465
 Else (feature 9 > 3.8175)
 If (feature 6 <= 16.5)
 Predict: 5.10763768115942
 Else (feature 6 > 16.5)
 Predict: 3.3010434782608686
 Else (feature 1 > 17.5)
 If (feature 3 <= 3.80833333325)
 If (feature 6 <= 8.5)
 If (feature 2 <= 198.583333325)
 Predict: 0.30301
 Else (feature 2 > 198.583333325)
 Predict: 0.6930606060606064
 Else (feature 6 > 8.5)
 If (feature 9 <= 4.6884999999999994)
 Predict: 3.149854368932038
 Else (feature 9 > 4.6884999999999994)
 Predict: 5.10964197530864
 Else (feature 3 > 3.80833333325)
 If (feature 5 <= 2.5)
 If (feature 6 <= 9.5)
 Predict: 0.8379480519480519
 Else (feature 6 > 9.5)
 Predict: 3.8957563559322033
 Else (feature 5 > 2.5)
 If (feature 3 <= 5.40833333325)
 Predict: 4.464601418439716
 Else (feature 3 > 5.40833333325)
 Predict: 5.82210507505361
 Else (feature 8 > 39.5)
 If (feature 9 <= 1.58)
 If (feature 1 <= 0.5)
 If (feature 6 <= 7.5)
 If (feature 8 <= 46.5)
 Predict: 7.976513098464318E-4
 Else (feature 8 > 46.5)
 Predict: 2.644466209598435E-5
 Else (feature 6 > 7.5)
 If (feature 9 <= 0.316)
 Predict: 0.058258855585831076
 Else (feature 9 > 0.316)
 Predict: 0.7020778443113775
 Else (feature 1 > 0.5)
 If (feature 6 <= 8.5)
 If (feature 8 <= 42.5)
 Predict: 0.0946923076923077
 Else (feature 8 > 42.5)
 Predict: 0.22543859649122802
 Else (feature 6 > 8.5)
 If (feature 6 <= 9.5)
 Predict: 1.1541226415094339
 Else (feature 6 > 9.5)
 Predict: 1.698404255319149
 Else (feature 9 > 1.58)
 If (feature 5 <= 10.5)
 If (feature 1 <= 19.5)
 If (feature 6 <= 15.5)
 Predict: 2.912131313131313
 Else (feature 6 > 15.5)
 Predict: 1.3514268292682927
 Else (feature 1 > 19.5)
 If (feature 1 <= 49.5)
 Predict: 3.7948125000000004
 Else (feature 1 > 49.5)
 Predict: 4.616976076555023
 Else (feature 5 > 10.5)
 If (feature 1 <= 25.5)
 If (feature 0 <= 0.102469136)
 Predict: 2.045456852791878
 Else (feature 0 > 0.102469136)
 Predict: 0.9974285714285754
 Else (feature 1 > 25.5)
 If (feature 6 <= 14.5)
 Predict: 3.6932808219178086
 Else (feature 6 > 14.5)
 Predict: 1.9126923076923084
 Tree 1 (weight 1.0):
 If (feature 6 <= 8.5)
 If (feature 9 <= 0.0695)
 If (feature 1 <= 0.5)
 If (feature 6 <= 6.5)
 If (feature 9 <= 5.0E-4)
 Predict: 5.410778071129056E-4
 Else (feature 9 > 5.0E-4)
 Predict: 0.19833333333333333
 Else (feature 6 > 6.5)
 If (feature 9 <= 5.0E-4)
 Predict: 0.02727062228654124
 Else (feature 9 > 5.0E-4)
 Predict: 0.2809875
 Else (feature 1 > 0.5)
 If (feature 6 <= 6.5)
 If (feature 5 <= 7.5)
 Predict: 0.05911711711711712
 Else (feature 5 > 7.5)
 Predict: 0.006405797101449278
 Else (feature 6 > 6.5)
 If (feature 9 <= 0.0115)
 Predict: 0.12233865814696486
 Else (feature 9 > 0.0115)
 Predict: 0.47547297297297303
 Else (feature 9 > 0.0695)
 If (feature 9 <= 0.316)
 If (feature 9 <= 0.15899999999999997)
 If (feature 8 <= 32.5)
 Predict: 0.5976241134751773
 Else (feature 8 > 32.5)
 Predict: 0.8142884615384616
 Else (feature 9 > 0.15899999999999997)
 If (feature 8 <= 19.5)
 Predict: 1.3537543859649124
 Else (feature 8 > 19.5)
 Predict: 0.9242403100775193
 Else (feature 9 > 0.316)
 If (feature 2 <= 151.0000000075)
 If (feature 5 <= 3.5)
 Predict: 2.3840909090909093
 Else (feature 5 > 3.5)
 Predict: 1.6486315789473682
 Else (feature 2 > 151.0000000075)
 If (feature 5 <= 3.5)
 Predict: 2.1491999999999996
 Else (feature 5 > 3.5)
 Predict: 1.3817410714285716
 Else (feature 6 > 8.5)
 If 

#### <b style="color:blue;"> Regression with gradient boosted trees</b>
[PySpark Reference](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.regression.GBTRegressor.html#pyspark.ml.regression.GBTRegressor)

In [0]:
gbt = GBTRegressor(featuresCol="features", labelCol="production", predictionCol="predicted_production", 
                           seed=42, 
                           maxDepth=2) 

model_name = "gbt" # for DF

In [0]:
pipeline_nolag = Pipeline(stages=[vec_assembler_nolag, gbt])
gbt_model_nolag = pipeline_nolag.fit(train_df)
gbt_prediction_nolag = gbt_model_nolag.transform(test_df)

In [0]:
#display(gbt_prediction.head(2))

In [0]:
gbt_rmse_nolag = evaluator.evaluate(gbt_prediction_nolag, {evaluator.metricName: "rmse"}) 
gbt_r2_nolag = evaluator.evaluate(gbt_prediction_nolag, {evaluator.metricName: "r2"}) 
#print('RMSE: ' + str(gbt_rmse_nolag)) 
#print('R^2: ' + str(gbt_r2_nolag))

newRow = spark.createDataFrame([[model_name, 'no lag', gbt_rmse_nolag, gbt_r2_nolag]], columns_DF)
evaluation_DF = evaluation_DF.union(newRow)

In [0]:
# with lag 1h
pipeline_wlag1h = Pipeline(stages=[vec_assembler_wlag1h, gbt])
gbt_model_wlag1h = pipeline_wlag1h.fit(train_df)
gbt_prediction_wlag1h = gbt_model_wlag1h.transform(test_df)
gbt_rmse_wlag1h = evaluator.evaluate(gbt_prediction_wlag1h, {evaluator.metricName: "rmse"}) 
gbt_r2_wlag1h = evaluator.evaluate(gbt_prediction_wlag1h, {evaluator.metricName: "r2"}) 
#print('RMSE: ' + str(gbt_rmse_wlag1h)) 
#print('R^2: ' + str(gbt_r2_wlag1h))

newRow = spark.createDataFrame([[model_name, 'with lag 1 hour', gbt_rmse_wlag1h, gbt_r2_wlag1h]], columns_DF)
evaluation_DF = evaluation_DF.union(newRow)

In [0]:
# with lag 24h
pipeline = Pipeline(stages=[vec_assembler_wlag24h, gbt])
gbt_model_wlag24h = pipeline_wlag24h.fit(train_df)
gbt_prediction_wlag24h = gbt_model_wlag24h.transform(test_df)
gbt_rmse_wlag24h = evaluator.evaluate(gbt_prediction_wlag24h, {evaluator.metricName: "rmse"}) 
gbt_r2_wlag24h = evaluator.evaluate(gbt_prediction_wlag24h, {evaluator.metricName: "r2"}) 
#print('RMSE: ' + str(gbt_rmse_wlag24h)) 
#print('R^2: ' + str(gbt_r2_wlag24h))

newRow = spark.createDataFrame([[model_name, 'with lag 24 hours', gbt_rmse_wlag24h, gbt_r2_wlag24h]], columns_DF)
evaluation_DF = evaluation_DF.union(newRow)

In [0]:
# with lags
pipeline = Pipeline(stages=[vec_assembler_wlags, gbt])
gbt_model_wlags = pipeline_wlags.fit(train_df)
gbt_prediction_wlags = gbt_model_wlags.transform(test_df)
gbt_rmse_wlags = evaluator.evaluate(gbt_prediction_wlags, {evaluator.metricName: "rmse"}) 
gbt_r2_wlags = evaluator.evaluate(gbt_prediction_wlags, {evaluator.metricName: "r2"}) 
#print('RMSE: ' + str(gbt_rmse_wlags))
#print('R^2: ' + str(gbt_r2_wlags))

newRow = spark.createDataFrame([[model_name, 'with lags', gbt_rmse_wlags, gbt_r2_wlags]], columns_DF)
evaluation_DF = evaluation_DF.union(newRow)

In [0]:
#gbt_tree = gbt_model.stages[-1]
#print(gbt_tree.featureImportances) # load feature importance from the model object

In [0]:
# Print the trees with nodes: 
#print('Trees with Nodes: {}'.format(gbt_tree.toDebugString))

In [0]:
#gbt_tree.toDebugString

#### <b style="color:blue;"> Compare models </b>

In [0]:
evaluation_DF = evaluation_DF.sort(col("RMSE"))
display(evaluation_DF)

Model,Input Data,RMSE,R^2
rf,with lags,0.7632212374971509,0.9312513465553528
gbt,with lags,0.7632212374971509,0.9312513465553528
rf,with lag 1 hour,0.77186198480283,0.9296858701271895
gbt,with lag 1 hour,0.7842017283304613,0.927419677556005
rf,with lag 24 hours,1.054566067595643,0.8687465011910007
gbt,with lag 24 hours,1.054566067595643,0.8687465011910007
gbt,no lag,1.183384442728275,0.8347220132262718
rf,no lag,1.250133991989478,0.8155509464056601


In [0]:
evaluation_DF.describe()

Out[69]: DataFrame[summary: string, Model: string, Input Data: string, RMSE: string, R^2: string]

[Random Forest vs GBT](https://subscription.packtpub.com/book/big-data-and-business-intelligence/9781788479042/1/ch01lvl1sec15/boosting-the-performance-using-random-forest-regressor):
- GBTs train one tree at a time, but Random Forest can train multiple trees in parallel. So the training time is lower for RF. However, in some special cases, training and using a smaller number of trees with GBTs is easier and quicker.
- RFs are less prone to overfitting in most cases, so it reduces the likelihood of overfitting. In other words, Random Forest reduces variance with more trees, but GBTs reduce bias with more trees.
- Finally, Random Forest can be easier to tune since performance improves monotonically with the number of trees, but GBT performs badly with an increased number of trees.

-> selected RF model

#### <b style="color:blue;"> [Hyperparameter Tuning](https://spark.apache.org/docs/latest/ml-tuning.html) with selected input data </b>

In [0]:
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [5, 10]) \
    .addGrid(rf.minInstancesPerNode, [1, 2,3,4]) \
    .addGrid(rf.numTrees, [10, 20, 50, 100]) \
    .build()

crossval = CrossValidator(estimator=pipeline_wlag1h,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3,
                          seed = 42)  # use 3+ folds in practice



In [0]:
# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(train_df)

In [0]:
print(cvModel)

CrossValidatorModel_796217b4cd0f

In [0]:
cv_prediction = cvModel.transform(test_df)

In [0]:
rf_rmse_cv = evaluator.evaluate(cv_prediction, {evaluator.metricName: "rmse"}) 
rf_r2_cv = evaluator.evaluate(cv_prediction, {evaluator.metricName: "r2"}) 
print('RMSE: ' + str(rf_rmse_cv)) 
print('R^2: ' + str(rf_r2_cv)) 

RMSE: 0.7769580615710971
R^2: 0.9377250831173495

#### <b style="color:blue;"> Save model w/o vec_assembler_wlag1h </b>

Model rf <br>
Input Data with lag 1 hour<br>
RMSE 0.9807332651984103<br>
R^2 0.900775295397652

In [0]:
features = ['rr_SMA', 'ss_SMA', 'dd_SMA', 'ff_SMA', 'year', 'month', 'hour', 'quarter-of-year', 'week-of-year', 'lag_1h']

In [0]:
rf = RandomForestRegressor(featuresCol="features", labelCol="production", predictionCol="predicted_production", # predictionCol="predicted_production",   # FALLS NOTWENDIG, ÜBERALL ANPASSEN
                           maxDepth = 5, minInstancesPerNode = 1, numTrees = 10, #output after cross validation
                           seed=42 
                           ) 

evaluator = RegressionEvaluator(labelCol="production",  
                                predictionCol="predicted_production") # predictionCol="predicted_production")   # FALLS NOTWENDIG, ÜBERALL ANPASSEN

model_name = "rf" # for DF

In [0]:
# lag 1h
pipeline_wlag1h = Pipeline(stages=[VectorAssembler(inputCols=features, outputCol='features'), rf])
rf_model_wlag1h = pipeline_wlag1h.fit(train_df)
rf_prediction_wlag1h = rf_model_wlag1h.transform(test_df)

rf_rmse_wlag1h = evaluator.evaluate(rf_prediction_wlag1h, {evaluator.metricName: "rmse"}) 
rf_r2_wlag1h = evaluator.evaluate(rf_prediction_wlag1h, {evaluator.metricName: "r2"}) 
print('RMSE: ' + str(rf_rmse_wlag1h)) 
print('R^2: ' + str(rf_r2_wlag1h)) 

RMSE: 0.9813124171647583
R^2: 0.9006580705566889

In [0]:
# model_to_save = rf_model_wlags
mPath =  "/path/to/model/folder/rf_model_wlag1h/" # new name?
rf_model_wlag1h.write().overwrite().save(mPath)